<a href="https://colab.research.google.com/github/Mainakdeb/project-jurkat/blob/main/project-jurkat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work in progress

In [55]:
!pip install iterative-stratification

In [ ]:
!pip install pytorch-lightning

In [124]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from albumentations import *
import torchvision.transforms as transforms
from tqdm.notebook import tnrange, tqdm
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from torchvision import models 
import torch.nn as nn
import torch.optim as optim
import time
import pytorch_lightning as pl
import seaborn as sns


In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
!wget https://data.broadinstitute.org/bbbc/BBBC048/BBBC048v1.zip

In [ ]:
!wget https://data.broadinstitute.org/bbbc/BBBC048/Ground_truth.lst

In [ ]:
!unzip "/content/BBBC048v1.zip"

In [ ]:
!unzip "/content/CellCycle.zip"

In [ ]:
my_file = open("/content/Ground_truth.lst", "r")
content = my_file.read()
print(type(content))
print(content[0:200])

In [63]:
dataframe1 = pd.read_csv("/content/Ground_truth.lst",
                          header=None) 

In [64]:
df=dataframe1[0].str.split(expand=True)
df=df.sample(frac=1)
df

,0,1,2
33571,32226,4,./G1/44782_Ch4.ome.jpg
53373,54050,5,./G2/28244_Ch3.ome.jpg
61122,58850,5,./G2/4172_Ch3.ome.jpg
96372,87251,6,./S/940_Ch3.ome.jpg
19313,6640,4,./G1/30102_Ch6.ome.jpg
...,...,...,...
9263,16154,4,./G1/1960_Ch6.ome.jpg
47342,58885,5,./G2/1777_Ch6.ome.jpg
47667,66552,5,./G2/18420_Ch3.ome.jpg
51070,51746,5,./G2/24189_Ch4.ome.jpg


In [65]:
#generate csv

import os, csv

cell_df = pd.DataFrame(list())
cell_df.to_csv('cell_cycle.csv')

f=open("cell_cycle.csv",'r+')
w=csv.writer(f)

folders = ["Anaphase", "G1", "G2","Metaphase", "Prophase", "S", "Telophase"]
folders2 = ["G1","G2","S"]

for f in folders2:
  for path, dirs, files in os.walk("/content/CellCycle/"+f+"/"):
      for filename in files:
          w.writerow([path+filename, f])


In [66]:
c = pd.read_csv('./cell_cycle.csv', header=None)
c.columns = ["path","class_name"]
print(c.class_name.unique())
c

['G1' 'G2' 'S']


,path,class_name
0,/content/CellCycle/G1/44410_Ch6.ome.jpg,G1
1,/content/CellCycle/G1/49778_merged.jpg,G1
2,/content/CellCycle/G1/3853_Ch6.ome.jpg,G1
3,/content/CellCycle/G1/19185_merged.jpg,G1
4,/content/CellCycle/G1/27736_Ch3.ome.jpg,G1
...,...,...
126195,/content/CellCycle/S/13505_Ch3.ome.jpg,S
126196,/content/CellCycle/S/24533_merged.jpg,S
126197,/content/CellCycle/S/49426_merged.jpg,S
126198,/content/CellCycle/S/25806_Ch4.ome.jpg,S


In [67]:
#delete non merged rows
c_merged=c[c.path.str[-10:] == "merged.jpg"]
c_merged.to_csv("cell_merged.csv")

In [68]:
def image_to_nparray(path):
    i = cv2.imread(path)
    i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    return(np.array(i))

In [69]:
class C_Dataset(Dataset):
    """custom"""

    def __init__(self, csv, root_dir='',augmentation=None, transform=None):
        
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.transform = transform
        self.root_dir = root_dir
        self.d = csv
        self.augmentation = augmentation
        self.class_dict = {'G1':0,
                           'S': 1,
                           'G2':2}
        
    def __getitem__(self, idx): 

        smol_img_path = str(self.d.iloc[idx][1]) #image path column
        #print(smol_img_path)
        class_name = str(self.d.iloc[idx][2])  #class name column
        #print("class............",class_name)
        #print(class_name)
        full_img_path = self.root_dir + smol_img_path #+ '.png'

        #print(full_img_path)

        img_arr = image_to_nparray(full_img_path)

        if self.augmentation is not None:
            img  = self.augmentation(image = img_arr)
            img2 = img["image"]
            #print(img2)

        if self.transform is not None:
            img_ret = self.transform(img2)
            #print(img_ret)
        
        #return self.class_dict[str(class_name)], img_ret
        return {"x": img_ret,
                "y": torch.tensor(self.class_dict[str(class_name)])
        }


    def __len__(self):
      
        return (len(self.d))


In [70]:
data = C_Dataset(pd.read_csv('/content/cell_merged.csv'), 
                 '',
                 augmentation = Compose([ 
                                        #RandomBrightnessContrast( p=0.9),
                                        Rotate(limit=30, interpolation=1, border_mode=4, always_apply=True, p=0.9)
                                        ]),
                transform = transforms.Compose([    
                                                transforms.ToPILImage(),
                                                transforms.Resize((224,224)),
                                                transforms.CenterCrop((128,128)),
                                                transforms.ToTensor()
                                                ])
                )

In [ ]:
train_dl = torch.utils.data.DataLoader(
    data
    ,batch_size=64
    ,shuffle=True
    # ,pin_memory=True
    # ,num_workers=4
)

for data in tqdm(train_dl):
    print("size: ",data['x'][0].size())
    plt.imshow(data['x'][0].permute(1,2,0))
    plt.show()
    break

In [ ]:
fig, ax = plt.subplots(3,3, figsize = (15,15))
class_dict = {'G1':0,
              'S': 1,
              'G2':2}

for i in range (9):
  batch = next(iter(train_dl))
  image, label=batch['x'],batch['y']
  #print(label)
  ax.flat[i].imshow(image[0].permute(1,2,0))
  
  text = "Class: "+list(class_dict.keys())[int(label[0])]
  ax.flat[i].set_xlabel(text)

In [172]:
#create a subset
df = pd.read_csv('/content/cell_merged.csv')
df = df.sample(frac=1)
# iloc[row slicing, column slicing]
subset_df = df.iloc[0:5000, 1:]
subset_df.to_csv("cell_merged_subset.csv")

In [173]:
#create folds
df = pd.read_csv('/content/cell_merged_subset.csv')

df.loc[:,"kfold"] = -1

#df = df.sample(frac=1).reset_index(drop=True)

targets = df.drop("class_name", axis=1).values

NFOLDS = 10
mskf = MultilabelStratifiedKFold(n_splits = NFOLDS)
for fold, (trn, val) in enumerate(mskf.split(X=df, y = targets)):
    df.loc[val, "kfold"] = fold
    
df.to_csv("./train_targets_folds.csv", index=False)

In [174]:
train_csv = pd.read_csv('./train_targets_folds.csv')
train_csv.head()

,Unnamed: 0,path,class_name,kfold
0,19940,/content/CellCycle/G2/38978_merged.jpg,G2,4
1,26669,/content/CellCycle/S/47182_merged.jpg,S,6
2,14582,/content/CellCycle/G2/21078_merged.jpg,G2,7
3,30537,/content/CellCycle/S/36157_merged.jpg,S,5
4,5470,/content/CellCycle/G1/32666_merged.jpg,G1,3


In [175]:
#####################################
class_names = ['G1', 'S', 'G2']

'''Load a predefined model & 
   reset the fully connected layer to add dropout + 
   change the number of classes'''
torch.cuda.empty_cache()
model = models.resnet101(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, len(class_names)))

'''Load model to device'''
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

'''Set evaluation criterion and optimization algorithm'''
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.00001)

train_losses = []
valid_losses = []
train_accuracy = []
valid_accuracy = []

def train_model(model, criterion, optimizer, num_epochs):
  start_time = time.time()
  for epoch in range(1, num_epochs + 1):
    print('Epoch {}/{}'.format(epoch , num_epochs))
    print('-*-' * 5)
    
    # TRAINING LOOP
    train_loss = 0.0
    valid_loss = 0.0
    total_tr = 0.0
    correct_tr = 0.0
    model.train()
    for batch in train_dl:

      data, target=batch['x'].to(device), batch['y'].to(device)
      optimizer.zero_grad()
      # Forward Pass
      output = model(data)
      loss = criterion(output, target)
      # Backward Pass  
      loss.backward()
      # Update loss and accuracy
      optimizer.step()
      train_loss += loss.item() * data.size(0)
      _,predicted = torch.max(output.data, 1)
      total_tr += target.size(0)
      correct_tr += (predicted == target).sum().item()
        
    # VALIDATION LOOP
    model.eval()
    total_vl = 0.0
    correct_vl = 0.0
    for batch in train_dl: 
      data, target=batch['x'].to(device), batch['y'].to(device)
      # Forward Pass      
      output = model(data)   
      loss = criterion(output, target)       
      # Update loss and accuracy
      valid_loss += loss.item() * data.size(0)
      _,predicted = torch.max(output.data, 1)
      total_vl += target.size(0)
      correct_vl += (predicted == target).sum().item()
  
    # Determine and update loss averages
    train_loss = train_loss/len(train_dl.sampler)
    valid_loss = valid_loss/len(train_dl.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    # Determine and update mean accuracy
    tr_accuracy = (100 * correct_tr/total_tr)
    vl_accuracy = (100 * correct_vl/total_vl)
    train_accuracy.append(tr_accuracy)
    valid_accuracy.append(vl_accuracy)
    
    # Display stats for each epoch
    print('Training Loss: {:.4f} Validation Loss: {:.4f}'.format(train_loss, valid_loss))
    print('Training Accuracy: {:.4f} Validation Accuracy: {:.4f}'.format( tr_accuracy, vl_accuracy))

  time_elapsed = time.time() - start_time
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  return model
model = train_model(model, criterion, optimizer, num_epochs=30)

In [191]:
class Engine:
  #something wrong here
      def __init__(self, model, optimizer, device, scheduler, criterion):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.criterion=criterion

      def train(self, data_loader):
        train_loss = 0.0
        valid_loss = 0.0
        total_tr = 0.0
        correct_tr = 0.0
        self.model.train()
        for batch in tqdm(data_loader):##
          data, target=batch['x'].to(self.device), batch['y'].to(self.device)
          self.optimizer.zero_grad()
          # Forward Pass
          output = self.model(data)
          loss = self.criterion(output, target)
          # Backward Pass  
          loss.backward()
          # Update loss and accuracy
          self.optimizer.step()
          train_loss += loss.item() * data.size(0)
          _,predicted = torch.max(output.data, 1)
          total_tr += target.size(0)
          correct_tr += (predicted == target).sum().item()
        #self.scheduler.step(1.)
        return(train_loss/len(data_loader.sampler), 100*correct_tr/total_tr)

      def evaluate(self, data_loader):
        self.model.eval()
        final_loss=0
        for data in data_loader:
            #self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            #loss.backward()
            #self.optimizer.step()
            final_loss += loss.item()
        return(final_loss)


In [192]:
def run_training(fold, batch_size, lr, save_model=True, num_epochs = 1):

    val_fold=train_csv[train_csv['kfold'] == fold]
    train_fold=train_csv[train_csv['kfold'] != fold]
    
    
    train_dataset = C_Dataset(train_fold, 
                 '',
                 augmentation = Compose([ 
                                        #RandomBrightnessContrast( p=0.9),
                                        Rotate(limit=30, interpolation=1, border_mode=4, always_apply=True, p=0.9)
                                        ]),
                transform = transforms.Compose([    
                                                transforms.ToPILImage(),
                                                transforms.Resize((224,224)),
                                                transforms.CenterCrop((128,128)),
                                                transforms.ToTensor()
                                                ])
                )
    valid_dataset = C_Dataset(val_fold, 
                 '',
                 augmentation = Compose([ 
                                        #RandomBrightnessContrast( p=0.9),
                                        Rotate(limit=30, interpolation=1, border_mode=4, always_apply=True, p=0.9)
                                        ]),
                transform = transforms.Compose([    
                                                transforms.ToPILImage(),
                                                transforms.Resize((224,224)),
                                                transforms.CenterCrop((128,128)),
                                                transforms.ToTensor()
                                                ])
                )
    
    train_loader = torch.utils.data.DataLoader(
                    train_dataset
                    ,batch_size=batch_size
                    ,shuffle=True
                )

    val_loader = torch.utils.data.DataLoader(
                    valid_dataset
                    ,batch_size=32
                    ,shuffle=True
                )

    torch.cuda.empty_cache()
    model_ft = models.resnet101(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, 3))
    model = model_ft.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.00001)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.1,
                                                 patience=7, 
                                                 eps=1e-4, 
                                                 verbose=True)
    eng = Engine(model,
                 optimizer, 
                 device='cuda', 
                 scheduler = scheduler,
                 criterion=criterion)
    
    best_loss = 999
    early_stop_iter = 10
    early_stop_count=0

    for epoch in tnrange(num_epochs):
        train_loss, train_accuracy = eng.train(train_loader)
        #valid_loss = eng.evaluate(val_loader)
        print("train_loss: {:.4f} train_acc: {:.4f}".format(train_loss, train_accuracy))#, "val_loss:", valid_loss)
        #print(tot)
        # if valid_loss<best_loss:
        #     best_loss = valid_loss
        #     if save_model:
        #         torch.save(model.state_dict(), "model_"+str(fold)+".pth")
        # else:
        #     early_stop_count +=1
            
        # if early_stop_count>early_stop_iter:
        #     break
                    
    return(best_loss)
    #to do: accuracy

In [ ]:
best_losses = []

for fold in range(NFOLDS):
    best_losses.append(
        run_training(
            fold=fold,
            batch_size = 32,#best_params["batch_size"],
            lr = 0.0001,#best_params["lr"],
            num_epochs = 20
        )
    )

In [ ]:
data_sub = C_Dataset(pd.read_csv('/content/cell_merged_subset.csv'), 
                 '',
                 augmentation = Compose([ 
                                        #RandomBrightnessContrast( p=0.9),
                                        Rotate(limit=30, interpolation=1, border_mode=4, always_apply=True, p=0.9)
                                        ]),
                transform = transforms.Compose([    
                                                transforms.ToPILImage(),
                                                transforms.Resize((224,224)),
                                                transforms.CenterCrop((128,128)),
                                                transforms.ToTensor()
                                                ])
                )

train_dl_sub = torch.utils.data.DataLoader(
    data_sub
    ,batch_size=64
    ,shuffle=True
    # ,pin_memory=True
    # ,num_workers=4
)

In [85]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, 3))
model = model_ft.to(device)
model.load_state_dict(torch.load("model_0.pth"))

<All keys matched successfully>

In [ ]:
class_names =  ['G1', 'S', 'G2']
confusion_matrix = torch.zeros(len(class_names), len(class_names))

with torch.no_grad():
    for i, data in enumerate(train_dl_sub, 0):
        images, labels = data['x'], data['y']
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(labels.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

cm = confusion_matrix.numpy()
fig = plt.figure(figsize=(6.5,6),  dpi = 80)
ax = sns.heatmap(cm,
                 annot=True, 
                 cmap='Blues', 
                 xticklabels = class_names, 
                 yticklabels = class_names, 
                 fmt='g')
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [128]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
loss = loss(input, target)
loss

tensor(1.9511, grad_fn=<NllLossBackward>)

In [129]:
input

tensor([[-8.3267e-01, -2.1798e-01,  3.4201e-01,  4.9026e-04, -1.3770e+00],
        [-5.9818e-01,  3.2150e-01,  4.0551e-01,  1.6947e-01,  1.4110e+00],
        [-4.0916e-01, -1.9820e-01,  6.8735e-02,  3.3822e-01, -1.5905e-01]],
       requires_grad=True)

In [130]:
target

tensor([3, 0, 4])

In [134]:
#test_loss_fn
def loss_fn(targets, outputs):
        return nn.CrossEntropyLoss() (outputs, targets)
loss_fn(target, input)

tensor(1.9511, grad_fn=<NllLossBackward>)